##  Calcul de $Q_1, Q_2, et Q_3$

Nous avons les formules suivantes pour Q_1, Q_2, et Q_3 :

$$ Q_1 = \frac{P_1 \cdot (0.76 \cdot (S_1 - 0.01) + K_1 + t_1 \cdot 0.001) \cdot \left(\frac{T_1}{1450}\right)^2}{R \cdot \frac{Mg}{100}} $$

$$ Q_2 = \frac{P_2 \cdot (0.76 \cdot (S_2 - 0.01) + K_2 + t_2 \cdot 0.001) \cdot \left(\frac{T_2}{1450}\right)^2}{R \cdot \frac{Mg}{100}} $$

$$ Q_3 = \frac{P_3 \cdot (0.76 \cdot (S_3 - 0.01) + K_3 + t_3 \cdot 0.001) \cdot \left(\frac{T_3}{1450}\right)^2}{R \cdot \frac{Mg}{100}} $$

Nous voulons que $ Q_1 = Q_2 = Q_3 $.

## Calcul du pourcentage de magnésium final dans le four de coulée

Pour garantir que le pourcentage de magnésium final dans le four de coulée est atteint malgré les pertes, nous établissons l'équation suivante :

$$ K_{\text{final}} \cdot{} (P_1 + P_2 + P_3) = K_1 \cdot{} P_1 + K_2 \cdot{} P_2 + K_3 \cdot{} P_3 + (Mg_{\text{perdu1}} + Mg_{\text{perdu2}}) $$

où :
- $ K_{\text{final}} \cdot{} (P_1 + P_2 + P_3) $ représente la quantité totale de magnésium nécessaire dans le four de coulée pour atteindre le pourcentage de magnésium final souhaité, multipliée par le poids total de la fonte.
- $ K_1 \cdot{} P_1 + K_2 \cdot{} P_2 + K_3 \cdot{} P_3 $ représente la quantité de magnésium résiduel initialement présente dans chaque lot de fonte.
- $ \text{mg\_perdu\_1} + \text{mg\_perdu\_2} $ représente la quantité totale de magnésium perdu par évaporation pendant le traitement.

Après réécriture, on obtient :

$$ K_1 \cdot{} P_1 + K_2 \cdot{} P_2 + K_3 \cdot{} P_3 = K_{\text{final}} \cdot{} (P_1 + P_2 + P_3) - (Mg_{\text{perdu1}} + Mg_{\text{perdu2}}) $$

# Code

In [1]:
import os
import pandas as pd
import numpy as np
import math

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc

In [85]:
from sympy import symbols, Eq, solve

def calcul_K_res_et_longueur_fil(P1, P2, P3, K_visee, eC, temps_traitement, T, S, t, R, Mg, eP, masse_fil):
    """
    Calcule les quantités de magnésium résiduel et la longueur théorique du fil fourré à utiliser pour que le graphite soit sous forme sphéroïdal.
    Args:
    - P1, P2, P3 : Poids des différentes fontes à traiter en Kg
    - K_visee : Quantité cible de magnésium pour que le graphite soit sous forme sphéroïdal en %
    - eP : Quantité de magnésium perdue par minute de séjour de la fonte après traitement ( phénomène d'évanouissement ) dans la poche en %
    - eC : Quantité de magnésium perdue par minute de séjour de la fonte après traitement ( phénomène d'évanouissement ) dans le four de coullée en %
    - temps_traitement : Durée du traitement en minutes
    - T : Température de la fonte au moment du traitement en degrés Celsius (même pour T1, T2, T3)
    - S : Taux de soufre de la fonte de base en %
    - t : Temps de séjour prévu pour la fonte après traitement en minutes
    - R : Rendement en magnésium de l'opération en %.
    - Mg : Taux en magnésium dans l'alliage en %.
    - masse_fil : Masse métrique du fil en g/m
    Returns:
    - K1, K2, K3 : Quantités de magnésium résiduel nécessaires pour chaque lot de fonte en %
    """
    
    # Calcul des pertes de magnésium
    mg_perdu = temps_traitement * eC

    
    S1, S2, S3 = S, S, S
    T1, T2, T3 = T, T, T
    t1, t2, t3 = t, t, t


    # Déclaration des variables symboliques
    K1, K2, K3 = symbols('K1 K2 K3')
    Q_1 = P1 * (0.76*(S1 - 0.01) + K1 + t1 * eP) * ((T1 / 1450)**2) / (R * Mg / 100)
    Q_2 = P2 * (0.76*(S2 - 0.01) + K2 + t2 * eP) * ((T2 / 1450)**2) / (R * Mg / 100)
    Q_3 = P3 * (0.76*(S3 - 0.01) + K3 + t3 * eP) * ((T3 / 1450)**2) / (R * Mg / 100)


    # Définition des équations
    eq1 = Eq(Q_1,Q_2)
    eq2 = Eq(Q_1,Q_3)
    eq3 = Eq(K_visee * (P1 + P2 + P3),
            K1*P1*( 1 - 2*mg_perdu) + K2*P2*(1 - mg_perdu) + K3*P3)


    
    # Résolution du système d'équations
    solution = solve((eq1, eq2, eq3), (K1, K2, K3))
    
    # Récupération des résultats
    K1_res = float(solution[K1])
    K2_res = float(solution[K2])
    K3_res = float(solution[K3])

    # Apres l'ajout des poches
    pct_mg_coulee1 = K1_res
    pct_mg_coulee2 = (pct_mg_coulee1*(1 - temps_traitement * eC)*P1 + K2_res*P2)/(P1 + P2)
    pct_mg_coulee3 = (pct_mg_coulee2*(1 - temps_traitement * eC)*(P1 + P2) + K3_res*P3)/(P1 + P2 + P3)
    

    # pct_mg_coulee1 = K1_res
    # pct_mg_coulee2 = (pct_mg_coulee1*(1 - temps_traitement * eC)*P1 + K2_res*P2)/((1 - pct_mg_coulee1*temps_traitement * eC)*P1 + P2)
    # pct_mg_coulee3 = (pct_mg_coulee2*(1 - temps_traitement * eC)*(P1 + P2) + K3_res*P3)/((1 - pct_mg_coulee2*temps_traitement * eC)*(P1 + P2) + P3)
    
    print(K1_res,K2_res,K3_res)
    print(pct_mg_coulee1,pct_mg_coulee2,pct_mg_coulee3)

    # Longueur de fil pour avoir la masse de Mg manquante
    Q = P1 * (0.76*(S1 - 0.01) + K1_res + t1 * eP) * ((T1 / 1450)**2) / (R * Mg / 100)
    L = Q / (masse_fil* 1e-3)   # en m
    return K1_res, K2_res, K3_res, pct_mg_coulee1, pct_mg_coulee2, pct_mg_coulee3, L


def export_result(df, dossier_data):
    """
    """
    # Créer le chemin complet du nouveau fichier Excel
    fichier_resultats = os.path.join(dossier_data, 'Resultats.xlsx')

    workbook = Workbook()
    feuille = workbook.active 

    # Écrire le DataFrame dans la feuille
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            feuille.cell(row=r_idx, column=c_idx, value=value)

    # Sauvegarder le classeur
    workbook.save(fichier_resultats)
    workbook.close()
    gc.collect()
    return 


def main_fct(chemin_fichier, dossier_courant):
    # Lecture de la première feuille du fichier Excel
    df = pd.read_excel(chemin_fichier, engine='openpyxl')

    # Supprimer les lignes vides
    df2 = df.dropna(how='all').reset_index(drop=True)

    # Extraction des paramètres Généraux de traitement GS
    pct_rendement_mg, masse_fil, masse_mg_fil, K_visee,pct_perdu_mg_coulee_min, pct_perdu_mg_poche_min, temps_traitement, temps_gs= [
        pd.to_numeric(df2.iloc[1, i], errors='coerce') for i in range(0, 8)
    ]
    # Extraction des variables du Fours de fusion
    pct_Soufre, tempera_fonte_poche = [
        pd.to_numeric(df2.iloc[4, i], errors='coerce') for i in range(3, 5)
    ]

    # Extraction des variables du Fours de couléee
    masse_fonte_poche_1, masse_fonte_poche_2, masse_fonte_poche_3 = [
        pd.to_numeric(df2.iloc[7, i], errors='coerce') for i in range(0, 3)
    ]
    pct_mg_fil = masse_mg_fil/masse_fil *100
    
    K1_res, K2_res, K3_res, pct_mg_coulee1, pct_mg_coulee2, pct_mg_coulee3, L = calcul_K_res_et_longueur_fil(masse_fonte_poche_1, masse_fonte_poche_2, masse_fonte_poche_3, 
                                                                K_visee, pct_perdu_mg_coulee_min, 
                                                                temps_traitement, tempera_fonte_poche, pct_Soufre, temps_gs, pct_rendement_mg, pct_mg_fil,pct_perdu_mg_poche_min, masse_fil)
            
    # Liste des noms de valeurs à rechercher
    output_name = [
        'Pourcentage de magnésium résiduel dans  la poche de traitement 1 (%)',
        'Pourcentage de magnésium résiduel dans  la poche de traitement 2 (%)',
        'Pourcentage de magnésium résiduel dans la poche de traitement 3 (%)',
        'Pourcentage de magnésium dans le four de coulée après l\'ajout poche 1 (%)',
        'Pourcentage de magnésium dans le four de coulée après l\'ajout poche 2 (%)',
        'Pourcentage de magnésium dans le four de coulée après l\'ajout poche 3 (%)',
        'Longueur théorique du fil fourré à utiliser (en m)'
    ]

    # Listes pour stocker les résultats
    indices_lignes_output = []
    noms_colonnes_output = []

    # Parcours du DataFrame pour trouver chaque valeur spécifique
    for target_value in output_name:
        found = False
        for index, row in df.iterrows():
            for col_name in df.columns:
                if row[col_name] == target_value:
                    indices_lignes_output.append(index)
                    noms_colonnes_output.append(col_name)
                    found = True
                    break
            if found:
                break

    df_res = df.copy()

    df_res.loc[indices_lignes_output[0] +1, noms_colonnes_output[0]] = K1_res
    df_res.loc[indices_lignes_output[1] +1, noms_colonnes_output[1]] = K2_res
    df_res.loc[indices_lignes_output[2]+1, noms_colonnes_output[2]] = K3_res
    df_res.loc[indices_lignes_output[3]+1, noms_colonnes_output[3]] = pct_mg_coulee1
    df_res.loc[indices_lignes_output[4]+1, noms_colonnes_output[4]] = pct_mg_coulee2
    df_res.loc[indices_lignes_output[5]+1, noms_colonnes_output[5]] = pct_mg_coulee3
    df_res.loc[indices_lignes_output[6]+1, noms_colonnes_output[6]] = L

    export_result(df_res, dossier_courant )
    return 



In [86]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'Transitoire.xlsm')
    # On recupere le chemin du dossier data
    dossier_courant = os.path.dirname(chemin_fichier)
    # Solve problème
    main_fct(chemin_fichier, dossier_courant)

0.0625517855946397 0.023286577331099984 0.10041466499162491
0.0625517855946397 0.034743180076854834 0.04500038707788115


On veut 0.0045% de Mg  dans P1 + P2 + P3 kg de fontes.
On perd 0.0005 % de Mg tout les 10 min entre chaque ajout de fontes P1, P2, P3 dans la coulée C avec respectivement K1, K2, K3 % de Mg.
Tu es mathématicien. Tu dois donnes une équation pour formaliser tous ca.
Fais le, si tu as des questions n'hésite pas à me demander !!



\begin{figure}[htbp]
    \centering
    \begin{tikzpicture}

    % Nodes with images
    \node (fusion) at (0,0) {\includegraphics[width=5cm,height=3cm]{Images/Fonderie_vue_de_bas.pdf}};
    \node (cabine) at (2,-5) {\includegraphics[width=5cm,height=3cm]{Images/Fonderie_vue_de_bas.pdf}};
    \node (coulee) at (12,-5) {\includegraphics[width=5cm,height=3cm]{Images/Fonderie_vue_de_bas.pdf}};

    % Labels for nodes
    \node[below=-3.6cm of fusion] {Fours de Fusion}; % below=-3.8cm of fusion
    \node[below=0cm of cabine] {Cabine de Fil fourré};
    \node[below=0cm of coulee] {Fours de Coulée};

    % Informational text
    \node[right=0.1cm of fusion, align=left] {F = 5000 kg};

    \node[left=0.1cm of cabine, align=left] {
        $P_1 = 1250 \text{ kg}$ \\
        $K_{\text{res1}} = 0.0533 $
    };

    \node[above=0.1cm of coulee, align=center] {
        $C_1 = 1250 \text{ kg}$ \\
        $K_{\text{final}} = 0.0533 $
    };

    % Arrows
    \draw[-{Latex[length=10mm,width=8mm]}, line width=4pt, gray, solid] (fusion) -- (cabine);
    \draw[-{Latex[length=10mm,width=8mm]}, line width=4pt, gray, solid] (cabine) -- (coulee);

    % Additional arrow
    % \draw[-{Latex[length=10mm,width=8mm]}, line width=4pt, gray, solid] (coulee) -- (fusion);

    \end{tikzpicture}
    \caption{Diagramme illustrant les processus de fusion et de coulée}
    \label{fig:fusion-coulee}
\end{figure}

On dispose de 4 images en png de ces 4 lieux:

Fours de Fusion :
F = 5000 kg

Cabine de Fil fourrée :
P_1 = 1250 kg
K_res1 = 0.0533

Poche de Coulée:
P_1 = 1250 kg
K_res1 = 0.049

Fours de Coulée :
C_1 = 1250 kg
K_final= 0.049


temps_gs = 4 min
temps_traitement = 10 min

Le temps de déplacement de la poche de traitement de la Cabine de Fil fourrée au Fours de coulée est de temps_gs min. Le temps de déplacement de la poche de traitement du Fours de Fusion au Fours de Coulée est de temps_traitement min.



Coulée dans les moulles:
C_1 = 1250 kg
K_final= 0.049

La poche de traitement contiste de la fonte et doit se déplacer dans ces 3 places. 
Elle recupère 1250 kg dans le Fours de Fusion puis va à la Cabine de Fil fourrée pour fixer le % de Mg résiduel à K_res1 = 0.0533 % dans la poche de traitement. Puis on verse la poche de traitement dans la poche de coulée. 


Tu es travaille dans une fonderie et tu dispose de 3 images en pdf de ces 3 lieux.
Tu dois illustrer ce processus, Peux-tu m'indiquer comment le faire sur lucidchart ? 
Si tu as des questions, n'hésite pas à me les poser !!

In [6]:
Quantite_mottes = 200
masse_grappe = 35 
nb_moules_heure = 130
Fontes_consomer = Quantite_mottes*masse_grappe
consommation_fonte_min = nb_moules_heure * masse_grappe / 60  # en kg/min

Fontes_coule_min = 2500
Fontes_coule = 3300



Fontes_fusion_dispo = 2000
temps_fusion = '1 hr 10 min 0 s'
pos_hr = temps_fusion.find('hr')
pos_min = temps_fusion.find('min')
pos_sec = temps_fusion.find('s')
# Extraire les valeurs numériques
now_hours = int(temps_fusion[:pos_hr].strip()) 
now_minutes = int(temps_fusion[pos_hr+3:pos_min].strip()) 
now_seconds = int(temps_fusion[pos_min+4:pos_sec].strip()) 
total_seconds = now_hours * 3600 + now_minutes * 60 + now_seconds


Fontes_coule_consommable = Fontes_coule - Fontes_coule_min
Fontes_dispo = (Fontes_fusion_dispo + Fontes_coule_consommable)

# En considérant que l'on consomme sans arret 
Temps_consommation = Fontes_dispo/consommation_fonte_min

if Fontes_dispo <= Fontes_consomer :
    # Lancer la fusion

    # Quand lancer la Fusion ?
    Fontes_produits = 5000 #5000*1 + 5000*1


total_seconds
Temps_consommation

36.92307692307693